In [1]:
import pandas as pd
import sys
sys.path.append("/Users/damoncrockett/ivpy/src")
from ivpy import *
from ivpy.extract import extract
from PIL import Image

In [2]:
df = pd.read_pickle('/Users/damoncrockett/lmlproc/dbdfgep.pkl')

In [3]:
roughs = df.loc[df.roughness.notnull()].index

In [4]:
df['impath'] = [f'/Users/damoncrockett/paperbase/src/assets/img/texture/{c}.jpg' for c in df.catalog]

In [5]:
#show(pathcol=df.impath.loc[roughs],thumb=8)

In [6]:
# sizes = []
# for i in df.index:
#     if i not in roughs:
#         sizes.append(None)
#     else:
#         im = Image.open(df.impath.loc[i])
#         sizes.append(im.size)

In [7]:
# df['imsize'] = sizes

In [8]:
# df['stdsize'] = df.imsize == (2448, 2048)

In [9]:
# tups = list(df.loc[roughs].loc[df.stdsize==False].imsize.unique())

In [10]:
# min([min(item) for item in tups])

In [11]:
# for item in tups:
#     if item[0]!=item[1]:
#         print(item)

In [12]:
def center_crop_image(image):
    # Calculate the center crop dimensions
    width, height = image.size
    new_width = 1016
    new_height = 1016
    
    left = (width - new_width)/2
    top = (height - new_height)/2
    right = (width + new_width)/2
    bottom = (height + new_height)/2
    
    # Crop the center of the image
    cropped_image = image.crop((left, top, right, bottom))
    return cropped_image

In [13]:
import os

In [14]:
# CROPDIR = "/Users/damoncrockett/paperbase/src/assets/neuralcrops/"

In [15]:
# for i in df.index:
#     if i in roughs:
#         impath = df.impath.loc[i]
#         im = Image.open(impath)
#         imcrop = center_crop_image(im)
#         fbase = os.path.basename(impath)
#         newpath = f'{CROPDIR}{fbase}'
#         imcrop.save(newpath)

In [16]:
# df['croppaths'] = [f'{CROPDIR}{df.catalog.loc[i]}.jpg' if i in roughs else None for i in df.index]

In [17]:
#montage(df.croppaths.loc[roughs],thumb=16,shape='rect')

In [18]:
BWDIR = "/Users/damoncrockett/paperbase/src/assets/neuralcropsbw/"

In [19]:
def desaturate_image(image):
    grayscale_image = image.convert('L')
    return grayscale_image

In [20]:
# for i in df.index:
#     if i in roughs:
#         impath = df.croppaths.loc[i]
#         im = Image.open(impath)
#         imbw = desaturate_image(im)
#         fbase = os.path.basename(impath)
#         newpath = f'{BWDIR}{fbase}'
#         imbw.save(newpath)

In [21]:
df['bwpaths'] = [f'{BWDIR}{df.catalog.loc[i]}.jpg' if i in roughs else None for i in df.index]

In [22]:
#montage(df.bwpaths.loc[roughs],thumb=16,shape='rect')

In [23]:
attach(df,'bwpaths')

In [24]:
# df.to_pickle('neural.pkl')

In [25]:
#X = extract('neural',verbose=False)

In [26]:
#X = X.applymap(lambda x: float(x) if x is not None else x)

In [27]:
#X.to_pickle('X.pkl')

# t-SNE

In [28]:
from ivpy.reduce import tsne

/Users/damoncrockett/lm/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [29]:
#df[['x','y']] = tsne(X.loc[roughs], init='pca')

In [30]:
# attach(df,'bwpaths')
# scatter('x','y',thumb=128,side=9000).save('/Users/damoncrockett/Desktop/xtsne.jpg')

In [31]:
# df['xbin'] = pd.cut(df.x,bins=100,labels=False)
# df['ybin'] = pd.cut(df.y,bins=100,labels=False)

In [32]:
# df.xbin = df.xbin.apply(lambda x: x-50)
# df.ybin = df.ybin.apply(lambda x: x-50)

In [33]:
#df.to_pickle('dftsnebins.pkl')

In [34]:
dfo = pd.read_pickle('dftsnebins.pkl')
dfo.xbin = dfo.xbin.apply(lambda x: x+50)
dfo.ybin = dfo.ybin.apply(lambda x: x+50)

In [35]:
dfo = dfo[['catalog','xbin','ybin']]

In [36]:
df = df.join(dfo.set_index('catalog'),'catalog')

In [37]:
del df['impath']
del df['bwpaths']
# del df['x']
# del df['y']

In [38]:
df['tsne'] = df.apply(lambda row: [row['xbin'], row['ybin']] if not pd.isna(row['xbin']) else None, axis=1)

In [39]:
del df['xbin']
del df['ybin']

In [40]:
df['tsne'] = df['tsne'].apply(lambda x: tuple(x) if isinstance(x,list) else x)

In [41]:
df = df.sort_values('tsne')

In [42]:
df['z'] = df.groupby('tsne').cumcount()

In [43]:
df['tsne'] = df['tsne'].apply(lambda x: list(x) if isinstance(x,tuple) else x)

In [44]:
df['tmap'] = df.apply(lambda row: [row['tsne'][0], row['tsne'][1], row['z']] if isinstance(row['tsne'],list) else None,
        axis=1)

In [45]:
del df['tsne']
del df['z']

In [46]:
df.to_pickle('/Users/damoncrockett/lmlproc/dbdfgepneural.pkl')